<a href="https://colab.research.google.com/github/ahmaadaziz/ai-face-crop/blob/main/ImageCrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python mtcnn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.3 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN

In [3]:
def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

In [4]:
# Function to resize an image while maintaining the aspect ratio
def resize_with_aspect_ratio(image, target_width=None, target_height=None):
    (h, w) = image.shape[:2]
    if target_width is None and target_height is None:
        return image

    if target_width is None:
        r = target_height / float(h)
        dim = (int(w * r), target_height)
    else:
        r = target_width / float(w)
        dim = (target_width, int(h * r))

    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    return resized

In [5]:
# Function to add padding around the image
def add_padding(image, padding_ratio=0.1):
    (h, w) = image.shape[:2]
    padding_h = int(h * padding_ratio)
    padding_w = int(w * padding_ratio)

    padded_image = cv2.copyMakeBorder(
        image, padding_h, padding_h, padding_w, padding_w, cv2.BORDER_CONSTANT, value=[0, 0, 0]
    )
    return padded_image

In [8]:
def process_face(image_path, output_path, gamma=1.5, target_width=256, padding_ratio=0.5):
    # Load the image
    img = cv2.imread(image_path)

    # Initialize MTCNN face detector
    detector = MTCNN()

    # Detect faces in the image
    result = detector.detect_faces(img)

    if result:
        # Extract bounding box from the first detected face
        bounding_box = result[0]['box']
        x, y, width, height = bounding_box

        # Calculate padding
        padding_x = int(width * padding_ratio)
        padding_y = int(height * padding_ratio)

        # Calculate new bounding box with padding
        new_x = max(0, x - padding_x)
        new_y = max(0, y - padding_y)
        new_width = min(img.shape[1] - new_x, width + 2 * padding_x)
        new_height = min(img.shape[0] - new_y, height + 2 * padding_y)

        # Crop the face from the image with padding
        cropped_face = img[new_y:new_y+new_height, new_x:new_x+new_width]

        # Apply gamma correction to the cropped face
        cropped_face = adjust_gamma(cropped_face, gamma)

        # Resize the cropped face while maintaining the aspect ratio
        cropped_face = resize_with_aspect_ratio(cropped_face, target_width=target_width)

        # Save the final processed image
        cv2.imwrite(output_path, cropped_face)
        print(f"Cropped and enhanced face saved to {output_path}")
    else:
        print("No face detected")

In [9]:
process_face('./image.jpg', './output_image1.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Cropped and enhanced face saved to ./output_image1.jpg
